<a href="https://colab.research.google.com/github/megaomics/Project1/blob/main/Protein%20aggregation%20articles1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q sentence-transformers faiss-cpu requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 140.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 107.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 127.1 MB/s eta 0:00:00


In [2]:
import requests

def search_articles(query, max_results=20):
    url = f"https://www.ebi.ac.uk/europepmc/webservices/rest/search?query={query}&format=json&pageSize={max_results}"
    response = requests.get(url)
    data = response.json()

    articles = []
    for result in data['resultList']['result']:
        articles.append({
            'title': result.get('title', 'No Title'),
            'abstract': result.get('abstractText', 'No Abstract'),
            'url': f"https://europepmc.org/article/{result.get('source')}/{result.get('id')}"
        })
    return articles

# 🔬 Search for articles related to protein aggregation
articles = search_articles("protein aggregation AND deep learning", max_results=30)

# Print first few results
for i, article in enumerate(articles[:3]):
    print(f"{i+1}. {article['title']}\n{article['abstract']}\n{article['url']}\n")


1. Recent advances in deep learning for protein-protein interaction: a review.
No Abstract
https://europepmc.org/article/MED/40524189

2. Network-based analyses of multiomics data in biomedicine.
No Abstract
https://europepmc.org/article/MED/40426270

3. RP3Net: a deep learning model for predicting recombinant protein production in<i>Escherichia coli</i>
No Abstract
https://europepmc.org/article/PPR/PPR1021627



In [3]:
from sentence_transformers import SentenceTransformer

# Load a pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Extract abstracts
abstracts = [a['abstract'] for a in articles]

# Convert to embeddings
embeddings = model.encode(abstracts, convert_to_tensor=False)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
import faiss
import numpy as np

# Convert embeddings to float32 numpy array
embedding_matrix = np.array(embeddings).astype('float32')

# Create FAISS index
dimension = embedding_matrix.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embedding_matrix)

# Example user query
query = "amyloid fibril formation in Alzheimer's disease"
query_embedding = model.encode([query], convert_to_tensor=False)
query_embedding = np.array(query_embedding).astype('float32')

# Search for top 5 most similar articles
D, I = index.search(query_embedding, k=5)

# Show top matching articles
print("\n🔍 Top Relevant Articles:")
for i in I[0]:
    print(f"📰 {articles[i]['title']}")
    print(f"🔗 {articles[i]['url']}\n")



🔍 Top Relevant Articles:
📰 Recent advances in deep learning for protein-protein interaction: a review.
🔗 https://europepmc.org/article/MED/40524189

📰 Network-based analyses of multiomics data in biomedicine.
🔗 https://europepmc.org/article/MED/40426270

📰 RP3Net: a deep learning model for predicting recombinant protein production in<i>Escherichia coli</i>
🔗 https://europepmc.org/article/PPR/PPR1021627

📰 Left-handed conformations of glycyl residues may confer protection against protein aggregation.
🔗 https://europepmc.org/article/MED/40243345

📰 AggNet: Advancing protein aggregation analysis through deep learning and protein language model.
🔗 https://europepmc.org/article/MED/39840791

